<a href="https://colab.research.google.com/github/IvanDePivan/2AMM30-groep-2-component-1/blob/main/Flairmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y torch
!pip install flair

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 34.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing 

In [2]:
import os
import random
import json
import re

file = {}
with open('annotated.json') as f:
    file = json.load(f)
    # print(file[1]['annotations'])


In [3]:
def write(filename, text):
  with open(f"{filename}.conll", "w") as file1:
    file1.write("\n\n".join(text))

In [4]:
# file_item = file[0]
def get_labels(file_item):
  rows = []
  position = 0
  next_label = 0
  # Sort the labels on start?
  labels = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'labels']
  labels.sort(key=lambda x: x['value']['start'])
  item_count = 1
  b_count = 0
  ids = {} # count -> id
  tokens = [item for item in re.split(r'(\s+)|([\(,\.\)\\;:$*&"\-–])|(\'\w+)|(\')', file_item['data']['text']) if item]
  for i in range(len(tokens)):
    item = tokens[i]
    if re.search(r'\s+', item):
      position += len(item)
      continue
    start = position
    end = position + len(item)
    row = f'{item_count}\t{item}'
    label = '\tO'


    if len(labels) > next_label:
      # if labels[next_label]["id"] == "bQJSunlNLq" or labels[next_label]["id"] == 'C2xT4AB2mw':
      #   print("#####################################################################")
      #   print(f'start: {start}, label_start: {labels[next_label]["value"]["start"]}')
      #   print(f'end: {end}, label_end: {labels[next_label]["value"]["end"]}')
      #   print(f'label: "{labels[next_label]["value"]["text"]}", token: "{item}"')
      # in some cases, te label does not extend to the end of a word, so end >= label-end, but label should be applied

      if start >= labels[next_label]['value']['start']:
        # if end <= labels[next_label]['value']['end']:
          # the current item has a label, and might have a relation
          id = labels[next_label]["id"]
          ids[item_count] = id
          if end > labels[next_label]['value']['end']:
            difference = end - labels[next_label]['value']['end']
            end -= difference
            tokens.insert(i+1, item[-difference:])
            item = item[:-difference]
          if start == labels[next_label]['value']['start']:
            # Current Item is start of a label
            b_count += 1
            label = '\tB-' + labels[next_label]["value"]["labels"][0]

          else:
            # Current item is inside a label
            if '\tB-' + labels[next_label]["value"]["labels"][0] in rows[-1] or '\tI-' + labels[next_label]["value"]["labels"][0] in rows[-1]:
              label = '\tI-' + labels[next_label]["value"]["labels"][0]
            else:
              b_count += 1
              label = '\tB-' + labels[next_label]["value"]["labels"][0]

      if end >= labels[next_label]['value']['end']:
        next_label += 1


    row = row + label
    position += len(item)
    item_count += 1
    rows.append(row)
  if len(labels) != b_count:
    print(f'labels: {len(labels)}, B-tags: {b_count}')
    print(labels)
    print(ids)
    print(f'missing label: {[item for item in labels if item["id"] not in ids.values()]}')
    print(rows)
  return rows, ids


In [5]:
def get_relations(file_item, ids):
  keys = ids.keys()
  relations = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'relation']
  bio_relations = []
  found_relations = 0
  for relation in relations:
    from_ids = []
    to_ids = []
    rel_type = relation['labels'][0]
    for key in keys:
      if ids[key] == relation['from_id']:
        from_ids.append(key)
      if ids[key] == relation['to_id']:
        to_ids.append(key)
    from_ids = [int(i) for i in from_ids]
    from_ids.sort()
    to_ids = [int(i) for i in to_ids]
    to_ids.sort()
    if len(from_ids) > 0 and len(to_ids) > 0:
      found_relations += 1
      bio_relations.append(f'{from_ids[0]};{from_ids[-1]};{to_ids[0]};{to_ids[-1]};{rel_type}')
  if found_relations != len(relations):
    print(f'found relations: {found_relations}, actual relations: {len(relations)}')
    print(relations)
    print(bio_relations)
    print(keys)
    print(ids)
  return bio_relations

In [6]:
conllu_file = []
for item in file:
  doc = ''
  rows, ids = get_labels(item)
  relations = get_relations(item, ids)
  if relations and len(relations) > 0:
    doc += '# relations = ' + ('|'.join(relations))
  doc += '\n' + ('\n'.join(rows))
  conllu_file.append(doc)

print(len(conllu_file))
write('annotated', conllu_file)

77


In [7]:
with open('annotated.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  print(len(splits))
  splits = sorted(splits, key=lambda x: random.random())
  train = splits[:58]
  test = splits[58:68]
  dev = splits[68:78]

  write('train', train)
  write('test', test)
  write('dev', dev)

77


In [8]:
import torch

torch.cuda.is_available()

True

In [9]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {1: 'text', 2: 'ner'}
data_folder = ''

# corpus: Corpus = ColumnCorpus(data_folder, columns,
#                               train_file='annotated_trim.conll',
#                               test_file='annotated_trim.conll',
#                               dev_file='annotated_trim.conll')

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='dev.conll')
print(f'Corpus size: {len(corpus.train)}')
corpus.test[1].to_tagged_string('ner')

2023-10-18 07:10:53,248 Reading data from .
2023-10-18 07:10:53,250 Train: train.conll
2023-10-18 07:10:53,251 Dev: dev.conll
2023-10-18 07:10:53,252 Test: test.conll
Corpus size: 58


'Sentence[447]: "Sven Anders Hedin , KNO1kl RVO , ( 19 February 1865 – 26 November 1952 ) was a Swedish geographer , topographer , explorer , photographer , travel writer and illustrator of his own works . During four expeditions to Central Asia , he made the Transhimalaya known in the West and located sources of the Brahmaputra , Indus and Sutlej Rivers . He also mapped lake Lop Nur , and the remains of cities , grave sites and the Great Wall of China in the deserts of the Tarim Basin . In his book " Från pol till pol " ( " From Pole to Pole " ) , Hedin describes a journey through Asia and Europe between the late 1880s and the early 1900s . While traveling , Hedin visited Turkey , the Caucasus , Tehran , Iraq , lands of the Kyrgyz people and the Russian Far East , India , China and Japan . The posthumous publication of his " Central Asia Atlas " marked the conclusion of his life \'s work . Overview . At 15 years of age , Hedin witnessed the triumphal return of the Arctic explorer Adol

In [10]:
# tag to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)


2023-10-18 07:10:53,909 Computing label dictionary. Progress:


58it [00:00, 6276.79it/s]

2023-10-18 07:10:53,932 Dictionary created for label 'ner' with 6 values: Winner (seen 561 times), Date (seen 369 times), Nationality (seen 278 times), Prizetype (seen 56 times), Reason (seen 37 times)


In [13]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings, TransformerWordEmbeddings
from flair.models import SequenceTagger
from typing import List
# embedding_types : List[TokenEmbeddings] = [
#         WordEmbeddings('glove'),
        ## other embeddings
        # ]
# embeddings : StackedEmbeddings = StackedEmbeddings(
#                                 embeddings=embedding_types)

embeddings = TransformerWordEmbeddings(
    model='distilbert-base-cased',
    layers="-1",
    subtoken_pooling='first',
    fine_tune=True,
    use_context=0,
    respect_document_boundaries=False,
)

tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,
        embeddings=embeddings,
        tag_dictionary=tag_dictionary,
        tag_type=tag_type,
        use_crf=False,
        use_rnn=False,
        reproject_embeddings=False,
        tag_format='BIO'
    )

2023-10-18 07:12:47,037 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason


In [14]:

# tagger : SequenceTagger = SequenceTagger(hidden_size=256,
#                                        embeddings=embeddings,
#                                        tag_dictionary=tag_dictionary,
#                                        tag_type=tag_type,
#                                        use_crf=True,
#                                          tag_format='BIO'
#                                          )
# print(tagger)

NameError: ignored

In [15]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
# trainer.train('resources/taggers/ner',
#               learning_rate=0.1,
#               mini_batch_size=32,
#               max_epochs=300) # 150

trainer.fine_tune(
        'resources/taggers/ner',
        learning_rate=5e-05,
        mini_batch_size=16,
        max_epochs=100,
        embeddings_storage_mode=None,
        weight_decay=0.0,
    )


2023-10-18 07:14:06,322 ----------------------------------------------------------------------------------------------------
2023-10-18 07:14:06,325 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28997, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_fea

100%|██████████| 1/1 [00:00<00:00,  1.06it/s]

2023-10-18 07:14:08,883 Evaluating as a multi-label problem: False
2023-10-18 07:14:08,915 DEV : loss 0.3801349997520447 - f1-score (micro avg)  0.5414
2023-10-18 07:14:08,932 ----------------------------------------------------------------------------------------------------


2023-10-18 07:14:09,498 epoch 2 - iter 1/4 - loss 0.38711576 - time (sec): 0.56 - samples/sec: 9720.45 - lr: 0.000006
2023-10-18 07:14:09,944 epoch 2 - iter 2/4 - loss 0.37292955 - time (sec): 1.01 - samples/sec: 10255.83 - lr: 0.000007
2023-10-18 07:14:10,494 epoch 2 - iter 3/4 - loss 0.38045333 - time (sec): 1.56 - samples/sec: 9755.75 - lr: 0.000008
2023-10-18 07:14:10,789 epoch 2 - iter 4/4 - loss 0.39353825 - time (sec): 1.86 - samples/sec: 9668.60 - lr: 0.000009
2023-10-18 07:14:10,797 ----------------------------------------------------------------------------------------------------
2023-10-18 07:14:10,799 EPOCH 2 done: loss 0.3935 - lr 0.000009


100%|██████████| 1/1 [00:01<00:00,  1.71s/it]

...

2023-10-18 07:19:03,636 Evaluating as a multi-label problem: False
2023-10-18 07:19:03,667 DEV : loss 0.5190640091896057 - f1-score (micro avg)  0.8305


2023-10-18 07:19:04,565 ----------------------------------------------------------------------------------------------------
2023-10-18 07:19:04,568 Testing using last state of model ...


100%|██████████| 1/1 [00:01<00:00,  1.89s/it]

2023-10-18 07:19:06,469 Evaluating as a multi-label problem: False
2023-10-18 07:19:06,498 0.7537	0.8836	0.8135	0.6856
2023-10-18 07:19:06,503 
Results:
- F-score (micro) 0.8135
- F-score (macro) 0.7353
- Accuracy 0.6856

By class:
              precision    recall  f1-score   support

      Winner     0.5847    0.8023    0.6765        86
 Nationality     0.9143    0.9143    0.9143        70
        Date     0.9545    1.0000    0.9767        63
   Prizetype     0.7000    1.0000    0.8235         7
      Reason     0.2500    0.3333    0.2857         6

   micro avg     0.7537    0.8836    0.8135       232
   macro avg     0.6807    0.8100    0.7353       232
weighted avg     0.7794    0.8836    0.8241       232

2023-10-18 07:19:06,504 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8134920634920635,
 'dev_score_history': [0.5413533834586466,
  0.544529262086514,
  0.5740740740740741,
  0.5949656750572082,
  0.6278026905829597,
  0.6403508771929824,
  0.691144708423326,
  0.6989010989010989,
  0.7079303675048356,
  0.7085714285714285,
  0.7344398340248962,
  0.7532467532467534,
  0.7705263157894738,
  0.7668845315904139,
  0.7612903225806452,
  0.7741935483870968,
  0.8113590263691685,
  0.8163265306122449,
  0.8008658008658008,
  0.7814569536423841,
  0.8134920634920635,
  0.7699316628701594,
  0.8063872255489022,
  0.7902869757174393,
  0.8230452674897119,
  0.8200836820083682,
  0.8297872340425532,
  0.808695652173913,
  0.8218623481781377,
  0.8172043010752688,
  0.8225469728601253,
  0.793859649122807,
  0.8024193548387096,
  0.8220338983050847,
  0.8170731707317073,
  0.8388520971302428,
  0.8050847457627118,
  0.822269807280514,
  0.8470066518847007,
  0.8326530612244899,
  0.8200836820083682,
  0.794759825327511,
  0.8154158215010141,
  0.

In [16]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the trained model
model = SequenceTagger.load("resources/taggers/ner/final-model.pt")

# create example sentence
sentence = Sentence('Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. ')

# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

2023-10-18 07:19:33,602 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason
Sentence[79]: "Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer." → ["Barack Hussein Obama II"/Winner, "August 4, 1961"/Date, "American"/Nationality, "the United States"/Nationality, "he"/Winner, "African-American"/Nationality, "Obama"/Winner, "U.S."/Nationality, "Illinois"/Nationality]


In [20]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import RelationExtractor

relation_label_dict = corpus.make_label_dictionary(label_type="relation")
results_base_path = 'resources/relations/'
relation_model: RelationExtractor = RelationExtractor(
        embeddings=embeddings,
        label_dictionary=relation_label_dict,
        label_type="relation",
        entity_label_type="ner",
        train_on_gold_pairs_only=True,
    )

trainer: ModelTrainer = ModelTrainer(relation_model, corpus)

trainer.train(
        results_base_path,
        learning_rate=0.1,
        mini_batch_size=2,
        max_epochs=10,
        shuffle=True,
    )

2023-10-18 07:38:44,424 Computing label dictionary. Progress:


58it [00:00, 7670.49it/s]

2023-10-18 07:38:44,441 Dictionary created for label 'relation' with 8 values: same_as (seen 220 times), received_nobelprize_in (seen 89 times), has_won (seen 83 times), has_nationality (seen 63 times), received_nobelprize_for (seen 60 times), born_on (seen 47 times), died_on (seen 24 times)
2023-10-18 07:38:44,493 ----------------------------------------------------------------------------------------------------
2023-10-18 07:38:44,496 Model: "RelationExtractor(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28997, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inp


/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-10-18 07:38:44,729 epoch 1 - iter 2/29 - loss 1.85191674 - time (sec): 0.21 - samples/sec: 227.57 - lr: 0.100000
2023-10-18 07:38:44,859 epoch 1 - iter 4/29 - loss 1.47210910 - time (sec): 0.34 - samples/sec: 249.98 - lr: 0.100000
2023-10-18 07:38:44,984 epoch 1 - iter 6/29 - loss 0.97665696 - time (sec): 0.46 - samples/sec: 288.58 - lr: 0.100000
2023-10-18 07:38:45,079 epoch 1 - iter 8/29 - loss 0.94820253 - time (sec): 0.56 - samples/sec: 246.21 - lr: 0.100000
2023-10-18 07:38:45,207 epoch 1 - iter 10/29 - loss 0.68593330 - time (sec): 0.68 - samples/sec: 277.63 - lr: 0.100000
2023-10-18 07:38:45,324 epoch 1 - iter 12/29 - loss 0.54710549 - time (sec): 0.80 - samples/sec: 298.12 - lr: 0.100000
2023-10-18 07:38:45,455 epoch 1 - iter 14/29 - loss 0.45003980 - time (sec): 0.93 - samples/sec: 312.20 - lr: 0.100000
2023-10-18 07:38:45,554 epoch 1 - iter 16/29 - loss 0.40010276 - time (sec): 1.03 - samples/sec: 319.04 - lr: 0.100000
2023-10-18 07:38:45,750 epoch 1 - iter 18/29 - loss 

...

2023-10-18 07:39:22,816 ----------------------------------------------------------------------------------------------------


100%|██████████| 5/5 [00:00<00:00,  8.72it/s]

2023-10-18 07:39:24,749 Evaluating as a multi-label problem: False


2023-10-18 07:39:24,795 0.0116	0.9855	0.023	0.0116
2023-10-18 07:39:24,797 
Results:
- F-score (micro) 0.023
- F-score (macro) 0.0396
- Accuracy 0.0116

By class:
                         precision    recall  f1-score   support

        has_nationality     0.0089    1.0000    0.0177        15
 received_nobelprize_in     0.0073    1.0000    0.0146        10
                born_on     0.0078    1.0000    0.0154         8
                same_as     0.0101    1.0000    0.0199        10
received_nobelprize_for     0.0218    1.0000    0.0428         9
                has_won     0.0518    1.0000    0.0985        10
                died_on     0.0355    0.8571    0.0682         7

              micro avg     0.0116    0.9855    0.0230        69
              macro avg     0.0205    0.9796    0.0396        69
           weighted avg     0.0193    0.9855    0.0374        69

2023-10-18 07:39:24,799 -----------------------------------------------------------------------------------------------

{'test_score': 0.023015738703672364,
 'dev_score_history': [0.02589667227761233,
  0.02537873883206008,
  0.026932539168716824,
  0.026932539168716824,
  0.026932539168716824,
  0.026932539168716824,
  0.026932539168716824,
  0.026932539168716824,
  0.026932539168716824,
  0.026932539168716824],
 'train_loss_history': [0.2652308753792037,
  0.027894730910520905,
  0.02769072913841526,
  0.001830633155727515,
  2.8345827978001445e-05,
  1.008196668842176e-05,
  7.618707364240227e-06,
  6.4542897349638096e-06,
  7.410697914065633e-06,
  7.508260101713879e-06],
 'dev_loss_history': [8.76096248626709,
  9.405643463134766,
  10.157445907592773,
  10.872288703918457,
  11.055466651916504,
  11.15264892578125,
  11.226800918579102,
  11.257524490356445,
  11.293221473693848,
  11.333389282226562]}

In [ ]:
file[0]['annotations'][0]['result']
# labels = [item for item in file[0]['annotations'][0]['result'] if item['type'] == 'labels']
relations = [item for item in file[0]['annotations'][0]['result'] if item['type'] == 'relation']
print(relations)

[{'type': 'relation', 'to_id': 'D_w-bbv0FW', 'labels': ['born_on'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': '6IEOyiTTRR', 'labels': ['died_on'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': '5SuvyzVQn4', 'labels': ['has_nationality'], 'from_id': 'qOz6CXctki', 'direction': 'right'}, {'type': 'relation', 'to_id': 'eoEtlIY3dq', 'labels': ['has_nationality'], 'from_id': 'WASMtd3xiq', 'direction': 'right'}, {'type': 'relation', 'to_id': '50aILF-wsY', 'labels': ['received_nobelprize_in'], 'from_id': 'lmkQq4uhNr', 'direction': 'right'}, {'type': 'relation', 'to_id': 'fDBIqiWAQ2', 'labels': ['has_won'], 'from_id': 'lmkQq4uhNr', 'direction': 'right'}, {'type': 'relation', 'to_id': 'Ytc4bT8tuJ', 'labels': ['has_nationality'], 'from_id': 'WASMtd3xiq', 'direction': 'right'}]


In [ ]:
loaded_model: RelationExtractor = RelationExtractor.load(results_base_path / "final-model.pt")
loaded_model.train_on_gold_pairs_only = False

sentence = Sentence(["Apple", "was", "founded", "by", "Steve", "Jobs", "."])
sentence[0:1].add_label("ner", "ORG")
sentence[4:6].add_label("ner", "PER")

loaded_model.predict(sentence)

sentence.get_labels("relation")[0].value

In [ ]:
labels[next_label]

{'id': 'qOz6CXctki',
 'type': 'labels',
 'value': {'end': 12,
  'text': 'Albert Camus',
  'start': 0,
  'labels': ['Winner']},
 'origin': 'manual',
 'to_name': 'text',
 'from_name': 'label'}

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/sota-ner-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  )

2023-10-16 15:04:21,946 SequenceTagger predicts: Dictionary with 21 tags: O, S-Winner, B-Winner, E-Winner, I-Winner, S-Date, B-Date, E-Date, I-Date, S-Nationality, B-Nationality, E-Nationality, I-Nationality, S-Prizetype, B-Prizetype, E-Prizetype, I-Prizetype, S-Reason, B-Reason, E-Reason, I-Reason
2023-10-16 15:04:21,974 ----------------------------------------------------------------------------------------------------
2023-10-16 15:04:21,978 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250003, 1024)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x XLMRobertaLayer(
    

OutOfMemoryError: ignored